<a href="https://colab.research.google.com/github/kasier48/DeepLearning/blob/main/Pratice_Week5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install langchain-community langchain-chroma langchain-openai bs4

Found existing installation: webdriver-manager 4.0.2
Uninstalling webdriver-manager-4.0.2:
  Successfully uninstalled webdriver-manager-4.0.2


In [8]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
import os, requests

openai_key = userdata.get('openai_key')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

# 2. GitHub Raw 파일 URL 설정
raw_url = "https://raw.githubusercontent.com/kasier48/DeepLearning/main/Week5/test.py"

# 3. HTTP GET 요청하여 파일 내용 가져오기
response = requests.get(raw_url)
if response.status_code == 200:
    code_content = response.text
    print("코드 내용을 성공적으로 가져왔습니다.")
    print(code_content[:500])
else:
    print(f"파일을 가져오는 데 실패했습니다. 상태 코드: {response.status_code}")
    print(response.text)

# 문서 객체 생성
docs = [Document(page_content=code_content)]

for idx, doc in enumerate(docs):
  print(f"idx; {idx + 1}, content: {doc.page_content}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=openai_key)
)

retriever = vectorstore.as_retriever()

user_msg = "코드를 리뷰하고 문제점을 말해줘."

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke(user_msg))

코드 내용을 성공적으로 가져왔습니다.
def Devide(a, b):
  return a / b

a = 10
b = 0

result = Devide(a, b)
print(result)
idx; 1, content: def Devide(a, b):
  return a / b

a = 10
b = 0

result = Devide(a, b)
print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


코드에서 'b'가 0일 때 나누기를 시도하므로 ZeroDivisionError가 발생합니다. 이를 방지하기 위해 나누기 전에 'b'가 0인지 확인하는 조건문을 추가해야 합니다. 따라서 예외 처리를 구현하는 것이 필요합니다.
